In [1]:
from google import genai
from google.genai import types
import os,re,json
import pathlib
import httpx
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [3]:
PROMPT_RAM = """
**Objective:** Extract data from Risk Assessment Matrix (RAM) tables within a provided document and output it into a structured CSV file.

**Phase 1: Identification of Relevant Risk Assessment Matrix Tables**
Carefully examine the document to identify the correct table(s) based on the following ordered criteria:

1.  **Primary Keyword Search:**
    * Look for titles or headings immediately preceding a table that contain any of these exact phrases (case-insensitive): "RAM", "Risk Assessment Matrix", "Overall Level of Concern".
    * Look for these same phrases within the table itself, either as a table heading or as a column header.
2.  **Handling Multi-Page Tables:**
    * If a table matching the criteria spans multiple pages, ensure all segments of the table are consolidated and treated as a single entity.
3.  **Appendix Search (Secondary Location):**
    * If no table is identified in the main body of the document using the primary keywords, search within any section labeled as "Appendix" (or similar, e.g., "Annex").
    * In the appendix, look for tables that contain column names such as (or similar to): "Source", "Risk", "Threat", "Likelihood", AND "Impact". The presence of columns related to both likelihood and impact is crucial here.
4.  **No Table Found:**
    * If, after following all the above steps, no suitable table can be confidently identified, return nothing or an empty CSV.

**Phase 2: Data Extraction and CSV Output Formatting**
Once a relevant table is identified, extract its data and structure it into a CSV file with the following precise columns. Apply these rules to each row of the identified table(s):

1.  **`Heading`**:
    * Content: Store the table's primary title or caption if available. If multiple (e.g., a number and a title), combine them. If no explicit title/caption is found directly associated with the table, leave this field empty for that table's rows.
2.  **`Risk Type`**:
    * Source: Identify the row, column or section headers  in the source table that most likely contains types of risks. They oftern appear as 'Global Risks','External Risks' or 'Domestic Risks','Country Specific Risks', or similar.
    * Content: it should be classificed as either "Global Risks" Or "Domestic Risks". This field cannot be empty. 
3.  **`Risk`**:
    * Source: Identify the column in the source table that most likely contains risk descriptions. Look for column headers containing keywords like "Risk", "Threat", "Hazard", or "Source".
    * Content: Extract the full text from this cell. This field cannot be empty. It often contains long, descriptive text.
4.  **`Time Horizon`**:
    * Source: Identify the column in the source table that most likely contains description on the time horizon of the risk. Look for column headers containing keywords like "Time Horizon".
    * Content: Extract coresponding time horizon description from this cell. There are cases where time horizon information is mix together with likelihood infomation, Make sure only extract time horizon infomation. This field is optional, if the table does not contain this information, leave it empty.
5.  **`Likelihood Full`**:
    * Source: Identify the column in the source table that most likely contains detailed likelihood information. Look for column headers containing the keyword "Likelihood".
    * Content: Extract EVERYTHING from this cell, including multiple paragraphs, bullet points, and any text separated by large blank spaces. This field cannot be empty.
6.  **`Likelihood`**:
    * Source: From the content extracted for `Likelihood Full`.
    * Content: Extract only the first line or primary categorical value that typically describes the level of likelihood (e.g., "High", "Medium", "Low", "Likely", "Unlikely", "Remote", "Possible", "Frequent"). If the first line is a detailed sentence, try to infer the single-word or short-phrase category.
7.  **`Impact Full`**:
    * Source: Identify the column in the source table that most likely contains detailed impact information. Look for column headers containing the keyword "Impact" or "Consequence".
    * Content: Extract EVERYTHING from this cell, including multiple paragraphs, bullet points, and any text separated by large blank spaces. This field cannot be empty.
8.  **`Impact`**:
    * Source: From the content extracted for `Impact Full`.
    * Content: Extract only the first line or primary categorical value that typically describes the level of impact (e.g., "High", "Medium", "Low", "Severe", "Moderate", "Minor", "Catastrophic", "Significant"). If the first line is a detailed sentence, try to infer the single-word or short-phrase category.
9.  **`Policy Response`**:
    * Source: Identify the column in the source table that most likely contains information about policy responses to the risk. Look for column headers containing keywords like "Policy", "Response", "Mitigation", "Action", or "Measure".
    * Content: Extract the full text from this cell. If no such column is found, or if the cell for a given risk is empty, leave this field empty in the CSV.

**Phase 3: Important Processing Details**

1.  **Footnote Removal:** If footnotes or source notes are present directly beneath the table content (and are clearly distinguishable from the main table data), they should be excluded from the extraction.
2.  **Data Integrity:**
    * If a row within an identified RAM table is missing data for the determined "Risk", "Likelihood Full", or "Impact Full" source columns, still include the row but leave the corresponding CSV cells empty for that missing piece (this slightly overrides "can't be empty" for cases where a source cell *within an otherwise valid RAM row* is blank). The expectation is that most rows in a valid RAM will have this data.
    * Prioritize accurate mapping of table columns to the requested CSV columns.
    * All data rows must have the **same number of columns** as the table header. Ensure all rows are well-formed and consistent.

**Output:**
Provide the extracted data as a single CSV file.

"""

In [5]:
def extract_csv_content(text):
    match = re.search(r"```csv(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def clean_markdown_fence(text):
    """
    Removes Markdown code fences like ```csv ... ``` and returns only the inner content.
    Handles cases with or without newlines, with or without language tags.
    """
    # Match fenced code block with or without language tag
    match = re.search(r"```(?:\w+)?(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return text.strip()

def clean_RAM(ram):

    letter_pattern = r'[a-zA-Z]'
    keyword_pattern = r'(risk|assessment|matrix|ram)'
    likelihood_impact_keywords = ['high', 'medium', 'low', 'moderate', 'severe']

    # Remove rows where 'Likelihood' or 'Impact' columns contain no letters

    cleaned_df = ram[
        ram['Likelihood'].str.contains(letter_pattern, na=False) &
        ram['Impact'].str.contains(letter_pattern, na=False) &
        ram['Likelihood Full'].str.contains(letter_pattern, na=False) &
        ram['Impact Full'].str.contains(letter_pattern, na=False)
    ]

    # Remove rows where 'Heading' does not contain RAM keywords if 'Heading' is not missing
    cleaned_df = cleaned_df[
        cleaned_df['Heading'].isna() | 
        (cleaned_df['Heading'].str.count(keyword_pattern, flags=re.IGNORECASE) >=2)
    ]

    # # Remove rows where 'Likelihood' or 'Impact' columns do not contain keywords from high, medium, low
    # cleaned_df = cleaned_df[
    #     cleaned_df['Likelihood'].str.contains('|'.join(likelihood_impact_keywords), case=False, na=False) |
    #     cleaned_df['Impact'].str.contains('|'.join(likelihood_impact_keywords), case=False, na=False)
    # ]

    # Keep only the specified columns
    cleaned_df = cleaned_df[['Heading', 'Risk Type','Risk', 'Time Horizon','Likelihood', 'Likelihood Full', 'Impact', 'Impact Full','Policy Response']]

    print(f"Rows before cleaning: {len(ram)}")
    print(f"Rows after cleaning: {len(cleaned_df)}")

    return cleaned_df

- process to convert to csv directly 

In [6]:
# Retrieve and encode the PDF byte
f_path= "/ephemeral/home/xiong/data/Fund/RAM_Table/RAM_Table_Examples/678_2023_0.pdf"
#### experimented with various coner cases 124_2022_0; 466_2022_0;  536_2023_0
filepath = pathlib.Path(f_path)
filepath.exists()

True

In [7]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])


In [8]:
df = pd.read_csv(StringIO(extract_csv_content(response.text)))
if df.empty:
    print("No relevant RAM table found.")
else:
    df = clean_RAM(df)

Rows before cleaning: 8
Rows after cleaning: 7


In [9]:
df

,Heading,Risk Type,Risk,Time Horizon,Likelihood,Likelihood Full,Impact,Impact Full,Policy Response
0,Annex VI. Risk Assessment Matrix,Global Risks,Intensification of regional conflict(s). Escal...,Shorter horizon (12-18 months),High,High,Medium,Medium\nFurther sanctions on Russia/Belorussia...,• Targeted social measures\n• Create fiscal sp...
1,Annex VI. Risk Assessment Matrix,Global Risks,Commodity price volatility. A succession of su...,Shorter horizon (12-18 months),Medium,Medium,High,High\nThe effect on Mali will depend on the si...,• Create fiscal space to scale up social spend...
2,Annex VI. Risk Assessment Matrix,Global Risks,Monetary policy miscalibration. Amid high econ...,Shorter horizon (12-18 months),Medium,Medium,Medium,Medium\nSupply shocks could cause core and hea...,• Create fiscal space for priority spending th...
3,Annex VI. Risk Assessment Matrix,Global Risks,Abrupt global slowdown or recession. Global an...,Shorter horizon (12-18 months),Medium,Medium,High,High\nMali will be negatively affected by lowe...,• Create fiscal buffers to absorb risk premia ...
5,Annex VI. Risk Assessment Matrix,Global Risks,Extreme climate events. Extreme climate events...,Longer horizon,Medium,Medium,Medium,Medium\nExtreme drought or rainfall could adve...,• Build resilience to climate change in agricu...
6,Annex VI. Risk Assessment Matrix,Domestic Risks,Significant deterioration of security situatio...,Shorter horizon (12-18 months),High,High,High,High\nA deterioration in the security situatio...,• Restore collaboration with international sec...
7,Annex VI. Risk Assessment Matrix,Domestic Risks,Risks of socio-political instability. Any dela...,Shorter horizon (12-18 months),Medium/High,Medium/High,High,High\nNew sanctions may constrain imports and ...,• Adhere to agreed timetable for return to dem...


- process to convert to markdown 

In [10]:
from IPython.display import Markdown, display

In [11]:
PROMPT_RAM_MD = """
**Objective:** Convert data from Risk Assessment Matrix (RAM) tables within a provided document and output it into a Markdown.

**Phase 1: Identification of Relevant Risk Assessment Matrix Tables**
Carefully examine the document to identify the correct table(s) based on the following ordered criteria:

1.  **Primary Keyword Search:**
    * Look for titles or headings immediately preceding a table that contain any of these exact phrases (case-insensitive): "RAM", "Risk Assessment Matrix", "Overall Level of Concern".
    * Look for these same phrases within the table itself, either as a table heading or as a column header.
2.  **Handling Multi-Page Tables:**
    * If a table matching the criteria spans multiple pages, ensure all segments of the table are consolidated and treated as a single entity.
3.  **Appendix Search (Secondary Location):**
    * If no table is identified in the main body of the document using the primary keywords, search within any section labeled as "Appendix" (or similar, e.g., "Annex").
    * In the appendix, look for tables that contain column names such as (or similar to): "Source", "Risk", "Threat", "Likelihood", AND "Impact". The presence of columns related to both likelihood and impact is crucial here.
4.  **No Table Found:**
    * If, after following all the above steps, no suitable table can be confidently identified, return nothing.
    
**Phase 2: Data Extraction and Markdown Output Formatting**
1.  Once a relevant table is identified, extract its data and structure it into a Markdown format.

**Phase 3: Important Processing Details**
1.  **Footnote Removal:** If footnotes or source notes are present directly beneath the table content (and are clearly distinguishable from the main table data), they should be excluded from the extraction.
2.  **Data Integrity:** the output content should be the exact content from the table, without any paraphrasing, summarization or modification in wording. 

**Output:**
Provide the extracted data as a Markdown file.
"""

In [12]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM_MD])

In [42]:
# print(response.text)

In [13]:
display(Markdown(clean_markdown_fence(response.text)))

### MALI
### Annex VI. Risk Assessment Matrix

| Source of Risk | Relative Likelihood | Expected Impact if Realized / Time Horizon | Policies to Mitigate Risks |
|---|---|---|---|
| **Global Risks / Conjunctural Shocks** ||||
| • Intensification of regional conflict(s). Escalation of Russia's war in Ukraine or other regional conflicts and resulting economic sanctions disrupt trade (e.g., energy, food, tourism, and/or critical supply chain components), remittances, refugee flows, FDI and financial flows, and payment systems. | High | Medium <br> Further sanctions on Russia/Belorussia could lead to supply shortages for fertilizer but is likely to have less of an impact on agricultural imports. Higher commodity prices could raise consumer price inflation. | • Targeted social measures <br> • Create fiscal space to scale up social spending <br> • Continue structural reforms to diversify the economy and remove constraints to growth <br> • Bolster trade links with alternative suppliers |
| • Commodity price volatility. A succession of supply disruptions (e.g., due to conflicts and export restrictions) and demand fluctuations (e.g., reflecting China reopening) causes recurrent commodity price volatility, external and fiscal pressures, and social and economic instability. | Medium | High <br> The effect on Mali will depend on the size and direction of any changes in different commodity prices. Higher food and energy commodity prices could cause social and economic tensions by raising Mali's food and energy imports bill, for example. Any volatility would potentially raise uncertainty, which could weigh on capital investment. | • Create fiscal space to scale up social spending <br> • Targeted social measures <br> • Raise domestic food supply and request aid from international partners for food/energy needs <br> • Review principles of the fiscal regime for mining taxation <br> • Enhance resilience against commodity price shocks by updating revenue projections in multi-year fiscal framework |
| • Monetary policy miscalibration. Amid high economic uncertainty and volatility, major central banks slow monetary policy tightening or pivot to loosen monetary policy stance prematurely, de-anchoring inflation expectations and triggering a wage-price spiral in tight labor markets. | Medium | Medium <br> Supply shocks could cause core and headline inflation to spike and lead to a further wage-price spiral, especially in the public sector. In addition, tighter global monetary policy could put downward pressure on gold prices, reduce gold production, investment and export revenues. Cotton prices may also fall in response to global recession. | • Create fiscal space for priority spending through revenue mobilization and gradual cuts in non-targeted subsidies, while protecting growth-enhancing capital spending as much as possible <br> • Avoid real increases in the public sector wage bill <br> • Longer term, diversify economy away from mining and the narrow agricultural base |
| • Abrupt global slowdown or recession. Global and idiosyncratic risk factors combine to cause a synchronized sharp growth downturn, with recessions in some countries, adverse spillovers through trade and financial channels, and markets fragmentation. <br> • EMDES: A new bout of global financial tightening, possibly combined with volatile commodity prices, leads to spiking risk premia, debt distress, widening of external imbalances, fiscal pressures, and sudden stops. | Medium | High <br> Mali will be negatively affected by lower gold and cotton prices, accounting for over 85 percent of its exports. A spike in risk premia on EMDE debt could lead to sovereign debt distress as the DSA already points to limited space to absorb shocks. | • Create fiscal buffers to absorb risk premia shocks <br> • Enhance resilience against commodity price shocks by updating revenue projections in multi-year fiscal framework <br> • Raise domestic food supply and request aid from international partners for food/energy needs |
| **Global Structural risks** ||||
| • Extreme climate events. Extreme climate events cause more severe than expected damage to infrastructure (especially in smaller vulnerable economies) and loss of human lives and livelihoods, amplifying supply chain disruptions and inflationary pressures, causing water and food shortages, and reducing growth. | Medium | Medium <br> Extreme drought or rainfall could adversely affect food production and livelihoods, particularly given that a large share of Mali's agriculture is for self-sustenance. | • Build resilience to climate change in agriculture <br> • Longer term, broaden the economic base and expand non-agricultural activities |
| **Domestic/Regional Risks** ||||
| • Significant deterioration of security situation. An increase in terrorist attacks across the country and a rise in the internally displaced population. | High | High <br> A deterioration in the security situation would be expected to lead to a rise in military spending, crowding out developmental priorities. Internal refugee flows would put emergency aid procedures under strain. | • Restore collaboration with international security forces and aid organizations <br> • Adhere to 2015 decentralization agreement with shared responsibilities |
| • Risks of socio-political instability. Any delay in the return to democratic rule beyond February 2024 could lead to new ECOWAS sanctions. Non-adherence to the 2015 agreement for decentralization could spark regional tensions in the north and central regions. | Medium/ High | High <br> New sanctions may constrain imports and freeze financial transactions, leading to heightened food insecurity and poverty. | • Adhere to agreed timetable for return to democratic rule <br> • Adhere to 2015 decentralization agreement with shared responsibilities |

### Experiment with full document without specific Table Identification 

In [44]:
full_doc_path= '/ephemeral/home/xiong/data/Fund/pdf_parse/Fund_Document/input/All_AIV_2008-2025_PDF/536_2023_0.pdf'
filepath = pathlib.Path(f_path)
filepath.exists()

True

In [45]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])

In [46]:
df = pd.read_csv(StringIO(extract_csv_content(response.text)))
if df.empty:
    print("No relevant RAM table found.")
else:
    df = clean_RAM(df)

Rows before cleaning: 7
Rows after cleaning: 7


In [50]:
df

,Heading,Risk Type,Risk,Time Horizon,Likelihood,Likelihood Full,Impact,Impact Full,Policy Response
0,Annex II. Risk Assessment Matrix¹,Global Risks,• EMDES: A new bout of global financial tighte...,NaN,Medium,Medium,High,"High. Lower GDP growth, due to weaker investme...",Use available fiscal space to provide targeted...
1,Annex II. Risk Assessment Matrix¹,Global Risks,Systemic financial instability 2/,NaN,Medium,Medium,Medium,Medium. An increase in capital outflows that l...,Provided the monetary policy stance remains ad...
2,Annex II. Risk Assessment Matrix¹,Global Risks,Commodity price volatility 3/,NaN,Medium,Medium,Medium,Medium. The economic impacts will critically d...,Use fiscal and monetary policy countercyclical...
3,Annex II. Risk Assessment Matrix¹,Global Risks,Deepening geo-economic fragmentation 4/,NaN,High,High,Medium,Medium. Weaker exports; reduced FDI inflows; i...,Accelerate structural reforms to facilitate th...
4,Annex II. Risk Assessment Matrix¹,Domestic Risks,"Persistently high domestic inflation, due to g...",NaN,Medium,Medium,High,High. Slower closing of the output gap amid we...,Use available fiscal space to provide targeted...
5,Annex II. Risk Assessment Matrix¹,Domestic Risks,Natural disasters Major natural disasters (e.g...,NaN,Medium,Medium,Low,Low. Disruption in economic activity in the af...,Prioritize expenditures to affected households...
6,Annex II. Risk Assessment Matrix¹,Domestic Risks,"A stronger boost from pent-up demand, together...",NaN,Medium,Medium,Medium,Medium. Stronger private sector investment as ...,Monetary policy can be tightened if inflation ...


In [48]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM_MD])
display(Markdown(clean_markdown_fence(response.text)))

# Annex II. Risk Assessment Matrix¹

| Category | Source of Risks | Likelihoo | Expected Impact | Policy Recommendation |
|---|---|---|---|---|
| **Global** | **Abrupt global slowdown or recession 1/** <br> • EMDES: A new bout of global financial tightening, possibly combined with volatile commodity prices, leads to spiking risk premia, debt distress, widening of external imbalances, fiscal pressures, and sudden stops. | Medium | **High.** Lower GDP growth, due to weaker investment and exports; a decline in capital inflows, leading to currency depreciation and tighter domestic credit conditions; higher poverty rate. | Use available fiscal space to provide targeted support, while allowing the exchange rate to act as a shock absorber. FXI could be used to address disorderly market conditions (e.g., a sharp pickup in the UIP premium), easing the burden on monetary policy. |
| **Global** | **Systemic financial instability 2/** | Medium | **Medium.** An increase in capital outflows that lead to large exchange rate depreciation and volatility; tighter financial conditions; and upward pressures on inflation through exchange rate passthrough. | Provided the monetary policy stance remains adequate to address inflation risks, and considering Indonesia's shallow FX markets, the use of FXI may be appropriate under certain circumstances (see Annex V for further discussions). |
| **Global** | **Commodity price volatility 3/** | Medium | **Medium.** The economic impacts will critically depend on whether commodity prices rise or decline, given Indonesia's commodity exporter status; A sharper-than-expected decline in international commodity prices would lead to lower growth, worsen the external balance, and lead to a faster moderation in inflation momentum. | Use fiscal and monetary policy countercyclically to stabilize output and inflation. Seek to make progress on reforming energy subsidies. The exchange rate should remain flexible and determined by market forces. |
| **Global** | **Deepening geo-economic fragmentation 4/** | High | **Medium.** Weaker exports; reduced FDI inflows; increased uncertainty leading to weaker investment. | Accelerate structural reforms to facilitate the transition to a higher value-added and greener economy, including through additional investment in education and infrastructure and by reducing trade restrictions. |
| **Domestic** | **Persistently high domestic inflation,** due to global or domestic supply disruptions, coupled with a slower-than-expected growth. | Medium | **High.** Slower closing of the output gap amid weaker private consumption and investment; a decline in capital inflows due to less positive economic prospects. | Use available fiscal space to provide targeted policy support to low-income households and small businesses. Monetary policy should carefully balance risks to output, inflation and financial stability and be tightened further if inflation surprises on the upside or if global monetary conditions tighten substantially, leading to large depreciation pressures on the rupiah. |
| **Domestic** | **Natural disasters** <br> Major natural disasters (e.g., volcanic eruptions and/or earthquakes) disrupt economic activity and affect sentiment, resulting in higher fiscal expenditures. | Medium | **Low.** Disruption in economic activity in the affected region; slower economic growth accompanied by a decline in portfolio inflows. | Prioritize expenditures to affected households and businesses. If the economy slows significantly, loosen monetary and fiscal policies to accelerate the recovery, strengthen monitoring of corporate vulnerabilities. |
| **Domestic** | **A stronger boost from pent-up demand,** together with a faster recovery in China, a faster fall in inflation domestically or in Indonesia's key trading partners can lift economic prospects | Medium | **Medium.** Stronger private sector investment as economic prospects improve, contributing to a faster-than-projected closing of the output gap; inflation stays high for longer; and the recovery in bank credit accelerates. | Monetary policy can be tightened if inflation momentum picks up. Macroprudential policies should be tightened if credit growth accelerates, closing the credit gap earlier. |

### process with json output 

In [21]:
# GenAI configuration
generation_config = {
    "temperature": 0.01,
    "top_p": 0.95,
    "top_k": 40,
    "max_output_tokens": 8000, # put it as 8k
    "response_mime_type": "application/json",
    }

In [23]:
response = client.models.generate_content(
          model="gemini-2.5-pro-preview-05-06",
          config=generation_config,
          contents=[
              types.Part.from_bytes(
                data=filepath.read_bytes(),
                mime_type='application/pdf',
              ),
              PROMPT_RAM])

In [24]:
response.text

'[\n  {\n    "Heading": "Annex VI. Risk Assessment Matrix",\n    "Risk Type": "Global Risks",\n    "Risk": "Intensification of regional conflict(s). Escalation of Russia\'s war in Ukraine or other regional conflicts and resulting economic sanctions disrupt trade (e.g., energy, food, tourism, and/or critical supply chain components), remittances, refugee flows, FDI and financial flows, and payment systems.",\n    "Time Horizon": "Shorter horizon (between 12 to 18 months)",\n    "Likelihood Full": "High",\n    "Likelihood": "High",\n    "Impact Full": "Medium\\nFurther sanctions on Russia/Belorussia could lead to supply shortages for fertilizer but is likely to have less of an impact on agricultural imports. Higher commodity prices could raise consumer price inflation.",\n    "Impact": "Medium",\n    "Policy Response": "• Targeted social measures\\n• Create fiscal space to scale up social spending\\n• Continue structural reforms to diversify the economy and remove constraints to growth\\